In [1]:
options(jupyter.plot_mimetypes = 'image/png')
set.seed(123)
library(TMB)
library(ggplot2)

Loading required package: Matrix


## GLM
- Specify distribution  
- Specify linear predictor  
- Specify link function

### example
- Discrete with Poisson, binomaial  
- Continous with lognormal, gamma

### class will cover
- Normal  
- Binomial  
- Poisson  

### MLE  
- $\hat{\theta} = argmax_{\theta}(L(\theta; y))$


In [2]:
setwd("~/Documents/Classes/2016_Spatio-temporal_models/Week 1 -- Likelihoods and linear models/Lab 1/")

In [4]:
library( SpatialDeltaGLMM )

#
data(WCGBTS_Canary_example)
CPUE = WCGBTS_Canary_example$HAUL_WT_KG
X = cbind( "Intercept"=rep(1,length(CPUE)) )


In [5]:
compile("delta_model_v1.cpp")
dyn.load( dynlib("delta_model_v1") )
Params = list("b_j"=rep(0,ncol(X)), "theta_z"=c(0,0))
Data = list( "y_i"=CPUE, "X_ij"=X )
Obj = MakeADFun( data=Data, parameters=Params, DLL="delta_model_v1")

[1] 0

In [8]:
Obj$fn( Obj$par )
Obj$gr( Obj$par )

[1] 7928.953

outer mgc:  3249 


-767.1285,-3249,-2201.7


In [9]:
Opt = nlminb( start=Obj$par, objective=Obj$fn, gradient=Obj$gr )

outer mgc:  3249 
outer mgc:  1783.454 
outer mgc:  529.8701 
outer mgc:  265.9077 
outer mgc:  112.5368 
outer mgc:  70.37939 
outer mgc:  19.60547 
outer mgc:  11.32693 
outer mgc:  7.148037 
outer mgc:  3.886266 
outer mgc:  2.42914 
outer mgc:  1.407305 
outer mgc:  0.7218772 
outer mgc:  0.1207936 
outer mgc:  0.01435134 
outer mgc:  0.001014479 
outer mgc:  6.423181e-05 


In [10]:
Opt$diagnostics = data.frame( "name"=names(Obj$par), "Est"=Opt$par, "final_gradient"=as.vector(Obj$gr(Opt$par)))

outer mgc:  6.423181e-05 


In [11]:
Opt$par # estimated parameters
SD = sdreport( Obj ) # standard errors

b_j   theta_z   theta_z 
1.3822136 2.5422402 0.5586374

outer mgc:  6.423181e-05 
outer mgc:  0.1815908 
outer mgc:  0.1815681 
outer mgc:  0.5142772 
outer mgc:  0.514749 
outer mgc:  1.108955 
outer mgc:  1.111046 


In [12]:
Report = Obj$report()

In [ ]:
# Step 2 -- build inputs and object
dyn.load( dynlib("delta_model_v1") )
Params = list("b_j"=rep(0,ncol(X)), "theta_z"=c(0,0))
Data = list( "y_i"=CPUE, "X_ij"=X )
Obj = MakeADFun( data=Data, parameters=Params, DLL="delta_model_v1")

# Step 3 -- test and optimize
Obj$fn( Obj$par )
Obj$gr( Obj$par )
Opt = nlminb( start=Obj$par, objective=Obj$fn, gradient=Obj$gr )
Opt$diagnostics = data.frame( "name"=names(Obj$par), "Est"=Opt$par, "final_gradient"=as.vector(Obj$gr(Opt$par)))
Opt$par # estimated parameters
SD = sdreport( Obj ) # standard errors

# Extract stuff
Report = Obj$report()

# Visualize fit
#png( file="Canary_histogram--with_fit.png", width=4, height=4, res=200, units="in")
  #par( mar=c(3,3,2,0), mgp=c(2,0.5,0), tck=-0.02)
  #hist( log(1+CPUE), freq=FALSE, col=rgb(1,0,0,0.2) )
  #Sim_CPUE = (1-rbinom(1e5, size=1, prob=Report$zero_prob)) * rlnorm(1e5, meanlog=Report$linpred_i, sdlog=Report$logsd)
  #hist( log(1+Sim_CPUE), freq=FALSE, add=TRUE, col=rgb(0,0,1,0.2) )
  #legend( "topright", bty="n", legend=c("Observed","Predicted"), fill=c("red","blue"))
#dev.off()
dyn.unload( dynlib("delta_model_v1") )

[1] 7928.953

outer mgc:  3249 


-767.1285,-3249,-2201.7


outer mgc:  3249 
outer mgc:  1783.454 
outer mgc:  529.8701 
outer mgc:  265.9077 
outer mgc:  112.5368 
outer mgc:  70.37939 
outer mgc:  19.60547 
outer mgc:  11.32693 
outer mgc:  7.148037 
outer mgc:  3.886266 
outer mgc:  2.42914 
outer mgc:  1.407305 
outer mgc:  0.7218772 
outer mgc:  0.1207936 
outer mgc:  0.01435134 
outer mgc:  0.001014479 
outer mgc:  6.423181e-05 
outer mgc:  6.423181e-05 


b_j   theta_z   theta_z 
1.3822136 2.5422402 0.5586374

outer mgc:  6.423181e-05 
outer mgc:  0.1815908 
outer mgc:  0.1815681 
outer mgc:  0.5142772 
outer mgc:  0.514749 
outer mgc:  1.108955 
outer mgc:  1.111046 


In [ ]:
# Step 0 -- make and compile template file
compile( "delta_model_v2.cpp" )

# Step 1 -- divide into partitions
K = 10
Partition_i = sample( x=1:K, size=length(CPUE), replace=TRUE )
PredNLL_k = rep(NA, K)

# Step 2 --Loop through partitions
for(k in 1:K){
  dyn.load( dynlib("delta_model_v2") )
  Params = list("b_j"=rep(0,ncol(X)), "theta_z"=c(0,0))
  Data = list( "y_i"=CPUE, "X_ij"=X, predTF_i=ifelse(Partition_i==k,1,0) )
  Obj = MakeADFun( data=Data, parameters=Params, DLL="delta_model_v2")
  Opt = nlminb( start=Obj$par, objective=Obj$fn, gradient=Obj$gr )
  Report = Obj$report()
  PredNLL_k[k] = Report$pred_jnll
}

# log-Predictive probability per datum
mean( PredNLL_k / table(Partition_i) )